In [29]:
import pandas as pd
import itertools, re, pickle, time

In [30]:
df = pd.DataFrame({
    'atc': ['antiinflamatorio', 'diabetico', 'malalt renal'], 
    'pas': [['ibuprofeno', 'aspirina'], ['insulina'], ['metformina', 'teramina']]})
df['word_count'] = df['pas'].apply(lambda x: len(x))


In [31]:
df

,atc,pas,word_count
0,antiinflamatorio,"[ibuprofeno, aspirina]",2
1,diabetico,[insulina],1
2,malalt renal,"[metformina, teramina]",2


ATC es una columna que conté una etiqueta que afegim a un grup de medicaments
PAS conté els principis actius de medicaments. 

L'objectiu és fer un script que donat un text qualsevol, em retorni les columnes ATC a les que pertany el text. Un *text* pertany a un grup només si tots els medicaments del PAS son presents. 

Textos de TEST amb el resultat que hauria de donar

* text1 = "li dóno aspirina" -> ATC: []
* text2 = "li dono aspirina i ibuprofeno -> ATC [ 'antiinflamatorio']
* text3 = "es una persona diabética" -> ATC []
* text4 = "pren metformina, teramina, insulina i ibuprofeno" -> ATC ['malalt renal', 'diabetico']


**Es presuposa que el text original ja ve una mica net... sense signes puntuacions, minúscules**


In [32]:
textos = [
    "li dóno aspirina",
    "li dono aspirina i ibuprofeno",
    "es una persona diabética",
    "pren metformina teramina insulina i ibuprofeno"
]



In [33]:
# PAS previ, que no cal optimitzar ja que es guarda a part, és un set amb els possibles principis actius, només per poder
# filtrar els textos per allò que estem buscant. 
pattern = re.compile('( \(|\)|[0-9]|p\.|l\n|)')
principios_activos = set()
for row in df['pas']:
    for item in row:
        for i in item.split(' '):
        #filter nums...
          i = i.lower().strip()
          principios_activos.add(pattern.sub('', i))
principios_activos

{'aspirina', 'ibuprofeno', 'insulina', 'metformina', 'teramina'}

In [34]:
def check_coincidences(df, text):
    pas = df['pas']
    coincidences = set([word.split(' ')[0] for word in pas]).intersection(text)
    return len(coincidences)

In [35]:
def add_group_atc(df, pas, text):
    #convert to list
    start_time = time.time()
    text = [text.lower().strip() for text in text.split(' ')]
    
    #clean text. Get only words into PAS
    valid_text = pas.intersection(text)
    #create new column that stores number of coincidences for each word
    df['coincidences'] = df.apply(check_coincidences, args=[valid_text], axis=1)
    
    # els que són d'un grup son els que tenen el número de coincidencies == número de medicaments
    intersection = df[df['coincidences'] == df['word_count']]['atc']
    
    # trec duplicats
    atcs = list(intersection.unique())
    
    # retorno el TEXT + els grups. 
    print('total time: ', time.time() - start_time)
    return ' '.join(text) + ' ---GRUPS:---' + ' '.join(atcs)

In [36]:
#text1: ha de 
for text in textos:
    print(add_group_atc(df, principios_activos, text))    

total time:  0.005441904067993164
li dóno aspirina ---GRUPS:---
total time:  0.0035779476165771484
li dono aspirina i ibuprofeno ---GRUPS:---antiinflamatorio
total time:  0.003171205520629883
es una persona diabética ---GRUPS:---
total time:  0.002465963363647461
pren metformina teramina insulina i ibuprofeno ---GRUPS:---diabetico malalt renal


# Carrega de les dades originals per veure el temps que triga a processar un text.

* df_genrics es el df amb el llistat de medicaments i grups
* pas: set de paraules mediques

Objectiu: millorar el total time!


In [37]:
df_generics = pd.read_pickle('./uploads/sire_generics.pkl')

In [38]:
with open('./uploads/principios_activos.pkl', 'rb') as file:
    pas = pickle.load(file)

In [39]:
print(add_group_atc(df_generics, pas, 'enalapril acetilsalicilico hola que tal warfarina dopidrogrel almodipino prindopril valsartan'))    

total time:  0.3863515853881836
enalapril acetilsalicilico hola que tal warfarina dopidrogrel almodipino prindopril valsartan ---GRUPS:---C09AA02 C09CA03 B01AC06
